In [2]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

In [3]:
df = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Desktop\ML\Day11\nutrient.csv", index_col =0)
df.head()

,energy,protein,fat,calcium,iron
Food_Item,,,,,
BEEF BRAISED,340,20,28,9,2.6
HAMBURGER,245,21,17,9,2.7
BEEF ROAST,420,15,39,7,2.0
BEEF STEAK,375,19,32,9,2.6
BEEF CANNED,180,22,10,17,3.7


In [4]:
# Create scaler: scaler
scaler = StandardScaler().set_output(transform='pandas')
dfscaled=scaler.fit_transform(df)

In [5]:
clust_DB = DBSCAN(eps=1, min_samples=2)
clust_DB.fit(dfscaled)
print(clust_DB.labels_)

[ 0  0 -1  0  1  1  1 -1  0  0  0  0  0  0  1  1 -1 -1  2  2  2  3  2  3
 -1  1 -1]


In [7]:
clust_df = df.copy()
clust_df["Clust"] = clust_DB.labels_
clust_df.sort_values(by='Clust')

,energy,protein,fat,calcium,iron,Clust
Food_Item,,,,,,
SHRIMP CANNED,110,23,1,98,2.6,-1
SARDINES CANNED,180,22,9,367,2.5,-1
BEEF ROAST,420,15,39,7,2.0,-1
CLAMS CANNED,45,7,1,74,5.4,-1
CLAMS RAW,70,11,1,82,6.0,-1
BEEF HEART,160,26,5,14,5.9,-1
PORK SIMMERED,355,19,30,9,2.4,0
PORK ROAST,340,19,29,9,2.5,0
SMOKED HAM,340,20,28,9,2.5,0


In [8]:
clust_df.groupby('Clust').mean()
clust_df.sort_values('Clust')

,energy,protein,fat,calcium,iron,Clust
Food_Item,,,,,,
SHRIMP CANNED,110,23,1,98,2.6,-1
SARDINES CANNED,180,22,9,367,2.5,-1
BEEF ROAST,420,15,39,7,2.0,-1
CLAMS CANNED,45,7,1,74,5.4,-1
CLAMS RAW,70,11,1,82,6.0,-1
BEEF HEART,160,26,5,14,5.9,-1
PORK SIMMERED,355,19,30,9,2.4,0
PORK ROAST,340,19,29,9,2.5,0
SMOKED HAM,340,20,28,9,2.5,0


In [9]:
dfscaled['Clust'] = clust_DB.labels_
df_scl_inliers = dfscaled[dfscaled['Clust']!=-1]
print( silhouette_score(df_scl_inliers.iloc[:,:-1],
                 df_scl_inliers.iloc[:,-1]) )

0.43236957939711584


In [10]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(dfscaled.iloc[:,:5])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            dfscaled['Clust'] = clust_DB.labels_
            df_scl_inliers = dfscaled[dfscaled['Clust']!=-1]
            sil_sc = silhouette_score(df_scl_inliers.iloc[:,:-1],
                             df_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]

0.2 2 0.5710837307456915
0.4 2 0.9040570858308878
0.6 2 0.7752441663222646
1 2 0.43236957939711584
1 3 0.43147150560271585
Best Paramters:


,Sr,eps,min_pt,sil
1,2.0,0.4,2.0,0.904057


In [13]:
clust_df = df.copy()
clust_df["Clust"] = clust_DB.labels_
clust_df.sort_values(by='Clust')

,energy,protein,fat,calcium,iron,Clust
Food_Item,,,,,,
BEEF TONGUE,205,18,14,7,2.5,-1
SARDINES CANNED,180,22,9,367,2.5,-1
SALMON CANNED,120,17,5,159,0.7,-1
PERCH FRIED,195,16,11,14,1.3,-1
MACKEREL CANNED,155,16,9,157,1.8,-1
MACKEREL BROILED,200,19,13,5,1.0,-1
HADDOCK FRIED,135,16,5,15,0.5,-1
CRABMEAT CANNED,90,14,2,38,0.8,-1
CLAMS CANNED,45,7,1,74,5.4,-1


In [15]:
clust_df.groupby('Clust').mean()
clust_df.sort_values('Clust')

,energy,protein,fat,calcium,iron,Clust
Food_Item,,,,,,
BEEF TONGUE,205,18,14,7,2.5,-1
SARDINES CANNED,180,22,9,367,2.5,-1
SALMON CANNED,120,17,5,159,0.7,-1
PERCH FRIED,195,16,11,14,1.3,-1
MACKEREL CANNED,155,16,9,157,1.8,-1
MACKEREL BROILED,200,19,13,5,1.0,-1
HADDOCK FRIED,135,16,5,15,0.5,-1
CRABMEAT CANNED,90,14,2,38,0.8,-1
CLAMS CANNED,45,7,1,74,5.4,-1


In [19]:
print(clust_df['Clust'].value_counts())

Clust
-1    20
 0     7
Name: count, dtype: int64
